# TransNeXt: Robust Foveal Visual Perception for Vision Transformers

In previous chapters, we revisited the strengths and weaknesses of convolutions and attention mechanisms, introduced CoAtNet—a model that combines these two paradigms—and explored ConvNeXt, a convolutional neural network inspired by the Vision Transformer (ViT) architecture. We also delved into Swin Transformer, which integrates a sliding window approach into the Vision Transformer framework. In this chapter, we will explore TransNeXt, a novel approach that enhances Vision Transformers with robust foveal visual perception.

## Introduction to TransNeXt

![TransNeXt](../imgs/TransNeXt.jpg)
Vision Transformers (ViTs) have revolutionized the field of computer vision by leveraging self-attention mechanisms to capture global context. However, ViTs often struggle with robustness and fine-grained details, which are critical for tasks requiring high precision. TransNeXt addresses these challenges by incorporating foveal visual perception principles, enhancing the robustness and detail sensitivity of ViTs.

## Advantages of TransNeXt

### Robustness

By emulating the human visual system, TransNeXt enhances the robustness of Vision Transformers. The model becomes more resilient to variations and distortions in the input data, improving performance on tasks with challenging conditions.

### Precision

The high-resolution foveal attention mechanism allows TransNeXt to focus on fine details, improving accuracy in tasks requiring detailed visual analysis. This makes TransNeXt particularly effective for applications such as medical imaging and fine-grained object recognition.

### Efficiency

TransNeXt achieves a balance between computational efficiency and performance. By dynamically adjusting the attention span, the model allocates resources more effectively, maintaining high performance without excessive computational overhead.

## Key Innovations in TransNeXt

### 1. High-Resolution Foveal Attention

TransNeXt introduces a high-resolution foveal attention mechanism, focusing on a smaller, more detailed region of the image. This is achieved by dynamically adjusting the attention span, allowing the model to allocate more computational resources to the regions requiring finer detail.

### 2. Low-Resolution Peripheral Attention

In parallel, TransNeXt employs a low-resolution peripheral attention mechanism, capturing the broader context of the image. This dual attention mechanism ensures that the model maintains a balance between detail and context, improving both precision and robustness.

### 3. Hierarchical Feature Maps

Similar to Swin Transformer, TransNeXt uses a hierarchical structure. The model progressively reduces the spatial dimensions of the feature maps while increasing the number of channels. This design allows TransNeXt to handle high-resolution images more efficiently and capture features at multiple scales.

### 4. Efficient Computation

By computing self-attention within local windows and progressively merging feature maps, TransNeXt achieves significant computational savings. This efficiency makes it feasible to apply the model to a wide range of vision tasks, from image classification to object detection and segmentation.

### 5. TransNeXt Block

The TransNeXt block is the fundamental building block of the model. Here's a breakdown of a typical TransNeXt block:
- **High-Resolution Attention**: Focuses on small, detailed regions.
- **Low-Resolution Attention**: Captures broader, contextual information.
- **Merging Mechanism**: Integrates the outputs from high and low-resolution attention mechanisms.

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
import numpy as np
from timm.models.layers import DropPath, to_2tuple, trunc_normal_

class DWConv(nn.Module):
    def __init__(self, dim=768):
        super(DWConv, self).__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=1, bias=True, groups=dim)

    def forward(self, x, H, W):
        B, N, C = x.shape
        x = x.transpose(1, 2).view(B, C, H, W).contiguous()
        x = self.dwconv(x)
        x = x.flatten(2).transpose(1, 2)
        return x

class ConvolutionalGLU(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super(ConvolutionalGLU, self).__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        hidden_features = int(2 * hidden_features / 3)
        self.fc1 = nn.Linear(in_features, hidden_features * 2)
        self.dwconv = DWConv(hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x, H, W):
        x, v = self.fc1(x).chunk(2, dim=-1)
        x = self.act(self.dwconv(x, H, W)) * v
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

@torch.no_grad()
def get_relative_position_cpb(query_size, key_size, pretrain_size=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pretrain_size = pretrain_size or query_size
    axis_qh = torch.arange(query_size[0], dtype=torch.float32, device=device)
    axis_kh = F.adaptive_avg_pool1d(axis_qh.unsqueeze(0), key_size[0]).squeeze(0)
    axis_qw = torch.arange(query_size[1], dtype=torch.float32, device=device)
    axis_kw = F.adaptive_avg_pool1d(axis_qw.unsqueeze(0), key_size[1]).squeeze(0)
    axis_kh, axis_kw = torch.meshgrid(axis_kh, axis_kw)
    axis_qh, axis_qw = torch.meshgrid(axis_qh, axis_qw)
    axis_kh = torch.reshape(axis_kh, [-1])
    axis_kw = torch.reshape(axis_kw, [-1])
    axis_qh = torch.reshape(axis_qh, [-1])
    axis_qw = torch.reshape(axis_qw, [-1])
    relative_h = (axis_qh[:, None] - axis_kh[None, :]) / (pretrain_size[0] - 1) * 8
    relative_w = (axis_qw[:, None] - axis_kw[None, :]) / (pretrain_size[1] - 1) * 8
    relative_hw = torch.stack([relative_h, relative_w], dim=-1).view(-1, 2)
    relative_coords_table, idx_map = torch.unique(relative_hw, return_inverse=True, dim=0)
    relative_coords_table = torch.sign(relative_coords_table) * torch.log2(
        torch.abs(relative_coords_table) + 1.0) / torch.log2(torch.tensor(8, dtype=torch.float32))
    return idx_map, relative_coords_table

class Attention(nn.Module):
    def __init__(self, dim, input_resolution, num_heads=8, qkv_bias=True, attn_drop=0., proj_drop=0.):
        super().__init__()
        assert dim % num_heads == 0, f"dim {dim} should be divided by num_heads {num_heads}."
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.temperature = nn.Parameter(torch.log((torch.ones(num_heads, 1, 1) / 0.24).exp() - 1))
        self.register_buffer("seq_length_scale", torch.as_tensor(np.log(input_resolution[0] * input_resolution[1])), persistent=False)
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.query_embedding = nn.Parameter(nn.init.trunc_normal_(torch.empty(self.num_heads, 1, self.head_dim), mean=0, std=0.02))
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)
        self.cpb_fc1 = nn.Linear(2, 512, bias=True)
        self.cpb_act = nn.ReLU(inplace=True)
        self.cpb_fc2 = nn.Linear(512, num_heads, bias=True)

    def forward(self, x, H, W, relative_pos_index, relative_coords_table):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, -1, 3 * self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        q, k, v = qkv.chunk(3, dim=1)
        rel_bias = self.cpb_fc2(self.cpb_act(self.cpb_fc1(relative_coords_table))).transpose(0, 1)[:, relative_pos_index.view(-1)].view(-1, N, N)
        attn = ((F.normalize(q, dim=-1) + self.query_embedding) * F.softplus(self.temperature) * self.seq_length_scale) @ F.normalize(k, dim=-1).transpose(-2, -1) + rel_bias
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

class Block(nn.Module):
    def __init__(self, dim, num_heads, input_resolution, window_size=3, mlp_ratio=4.,
                 qkv_bias=False, drop=0., attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, sr_ratio=1, fixed_pool_size=None):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, input_resolution, num_heads=num_heads, qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop)
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = ConvolutionalGLU(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x, H, W, relative_pos_index, relative_coords_table):
        x = x + self.drop_path(self.attn(self.norm1(x), H, W, relative_pos_index, relative_coords_table))
        x = x + self.drop_path(self.mlp(self.norm2(x), H, W))
        return x

### 6. TransNeXt Macro Architecture

The TransNeXt consists of multiple stages, each containing several TransNeXt blocks. The number of blocks and their configurations can be adjusted to create different variants of the model.

In [ ]:
class OverlapPatchEmbed(nn.Module):
    def __init__(self, patch_size=7, stride=4, in_chans=3, embed_dim=768):
        super().__init__()
        patch_size = to_2tuple(patch_size)
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=stride, padding=(patch_size[0] // 2, patch_size[1] // 2))
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        x = self.proj(x)
        _, _, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        x = self.norm(x)
        return x, H, W

class TransNeXt(nn.Module):
    def __init__(self, img_size=224, pretrain_size=None, window_size=[3, 3, 3, None], patch_size=16, in_chans=3, num_classes=1000, embed_dims=[64, 128, 256, 512], num_heads=[1, 2, 4, 8], mlp_ratios=[4, 4, 4, 4], qkv_bias=False, drop_rate=0., attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm, depths=[3, 4, 6, 3], sr_ratios=[8, 4, 2, 1], num_stages=4, fixed_pool_size=None):
        super().__init__()
        self.num_classes = num_classes
        self.depths = depths
        self.num_stages = num_stages
        pretrain_size = pretrain_size or img_size
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        cur = 0
        for i in range(num_stages):
            patch_embed = OverlapPatchEmbed(patch_size=7 if i == 0 else 3, stride=4 if i == 0 else 2, in_chans=in_chans if i == 0 else embed_dims[i - 1], embed_dim=embed_dims[i])
            num_patches = (pretrain_size // (4 if i == 0 else 2)) ** 2
            relative_pos_index, relative_coords_table = get_relative_position_cpb((pretrain_size // (4 if i == 0 else 2), pretrain_size // (4 if i == 0 else 2)), (pretrain_size // (4 if i == 0 else 2), pretrain_size // (4 if i == 0 else 2)))
            blocks = nn.ModuleList([Block(dim=embed_dims[i], num_heads=num_heads[i], input_resolution=(pretrain_size // (4 if i == 0 else 2), pretrain_size // (4 if i == 0 else 2)), window_size=window_size[i], mlp_ratio=mlp_ratios[i], qkv_bias=qkv_bias, drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[cur + j], norm_layer=norm_layer, sr_ratio=sr_ratios[i], fixed_pool_size=fixed_pool_size) for j in range(depths[i])])
            cur += depths[i]
            norm = norm_layer(embed_dims[i])
            setattr(self, f"patch_embed{i + 1}", patch_embed)
            setattr(self, f"blocks{i + 1}", blocks)
            setattr(self, f"norm{i + 1}", norm)
        self.head = nn.Linear(embed_dims[-1], num_classes) if num_classes > 0 else nn.Identity()
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        for i in range(self.num_stages):
            patch_embed = getattr(self, f"patch_embed{i + 1}")
            blocks = getattr(self, f"blocks{i + 1}")
            norm = getattr(self, f"norm{i + 1}")
            x, H, W = patch_embed(x)
            for blk in blocks:
                x = blk(x, H, W)
            x = norm(x)
            if i < self.num_stages - 1:
                x = x.permute(0, 2, 1).view(x.size(0), -1, H, W)
        x = x.mean(dim=1)
        x = self.head(x)
        return x


input_tensor = torch.randn(8, 3, 224, 224)  # Batch of 8, 3x224x224 images
model = TransNeXt()
output = model(input_tensor)
print(output.shape)  # Expected output shape: (8, 1000)
print(model)